In [1]:
################################################################################
# Load dataset and split it into training and test set
################################################################################

import pandas as pd
import os
from tabulate import tabulate
from sklearn.preprocessing import LabelEncoder

sample_size = 10000

# Load dateset
df = pd.read_csv(os.getcwd() + f'/data/sample-{sample_size}-2.csv')

# Encode categorical columns
label_encoder = LabelEncoder()
categorical_columns = df.select_dtypes(include=['object']).columns
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])

# Split dataset according to attack type
normal_df = df[df['Attack_label'] == 0]
attack_df = df[df['Attack_label'] == 1]

# Split dataset into training and test set
normal_df_train = normal_df.sample(frac=0.8, random_state=42)
normal_df_test = normal_df.drop(normal_df_train.index)
attack_df_train = attack_df.sample(frac=0.8, random_state=42)
attack_df_test = attack_df.drop(attack_df_train.index)

X_train = pd.concat([normal_df_train, attack_df_train]).drop(columns=['frame.time', 'Attack_label', 'Attack_type'])
y_train = pd.concat([normal_df_train, attack_df_train])['Attack_label']
X_test = pd.concat([normal_df_test, attack_df_test]).drop(columns=['frame.time', 'Attack_label', 'Attack_type'])
y_test = pd.concat([normal_df_test, attack_df_test])['Attack_label']

print("Training set size: ", X_train.shape[0])
print("Test set size: ", X_test.shape[0])
print(X_train.columns)

Training set size:  8000
Test set size:  2000
Index(['ip.src_host', 'ip.dst_host', 'arp.dst.proto_ipv4', 'arp.opcode',
       'arp.hw.size', 'arp.src.proto_ipv4', 'icmp.checksum', 'icmp.seq_le',
       'icmp.transmit_timestamp', 'http.file_data', 'http.content_length',
       'http.request.uri.query', 'http.request.method', 'http.referer',
       'http.request.full_uri', 'http.request.version', 'http.response',
       'tcp.ack', 'tcp.ack_raw', 'tcp.checksum', 'tcp.connection.fin',
       'tcp.connection.rst', 'tcp.connection.syn', 'tcp.connection.synack',
       'tcp.dstport', 'tcp.flags', 'tcp.flags.ack', 'tcp.len', 'tcp.options',
       'tcp.payload', 'tcp.seq', 'tcp.srcport', 'udp.port', 'udp.stream',
       'udp.time_delta', 'dns.qry.name', 'dns.qry.name.len', 'dns.qry.qu',
       'dns.retransmission', 'dns.retransmit_request', 'mqtt.conack.flags',
       'mqtt.conflag.cleansess', 'mqtt.conflags', 'mqtt.hdrflags', 'mqtt.len',
       'mqtt.msg', 'mqtt.msgtype', 'mqtt.proto_len', 'mq

In [2]:
################################################################################
# Predict from Decision Tree model
################################################################################

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Create an instance of the DecisionTreeClassifier model
model = DecisionTreeClassifier()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict the labels for the test data
y_true = y_test
y_pred = model.predict(X_test)

# Evaluate the model
c_report = classification_report(y_true, y_pred)
c_matrix = confusion_matrix(y_true, y_pred)

with open(f"results/result-dt-{sample_size}-2.txt", "w") as f:
    f.write(f"Classication Report\n{c_report}\n\nConfusion Matrix\n{c_matrix}")

print(c_report)
print(c_matrix)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1000
           1       1.00      1.00      1.00      1000

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

[[1000    0]
 [   0 1000]]


In [3]:
################################################################################
# Feature Importance - Decision Tree 
################################################################################

from sklearn.tree import DecisionTreeClassifier

n = 100

feature_importances = {}
for i in range(n):
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    sorted_features = sorted(zip(model.feature_importances_, model.feature_names_in_), reverse=True)
    for importance, name in sorted_features:
        if name in feature_importances:
            feature_importances[name].append(importance)
        else:
            feature_importances[name] = [importance]

average_feature_importances = {}
for name, importances in feature_importances.items():
    average_feature_importances[name] = sum(importances) / len(importances)

top_features = sorted(average_feature_importances.items(), key=lambda x: x[1], reverse=True)

with open(f"results/feature-importance-{sample_size}-dt.txt", "a") as f:
    f.write("\n".join([str(feature) for feature in top_features[:5]]))

print(top_features[:5])

[('dns.qry.name.len', 0.9979026217228476), ('mqtt.msg', 0.0004244694132334541), ('dns.retransmission', 0.00039950062421972144), ('mqtt.conack.flags', 0.00034956304619225624), ('mqtt.topic', 0.0003245942571785236)]


In [2]:
################################################################################
# Predict from Random Forest model
################################################################################

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Create an instance of the DecisionTreeClassifier model
model = RandomForestClassifier()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict the labels for the test data
y_true = y_test
y_pred = model.predict(X_test)

# Evaluate the model
c_report = classification_report(y_true, y_pred)
c_matrix = confusion_matrix(y_true, y_pred)

with open(f"results/result-rf-{sample_size}-2.txt", "w") as f:
    f.write(f"Classication Report\n{c_report}\n\nConfusion Matrix\n{c_matrix}")

print(c_report)
print(c_matrix)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1000
           1       1.00      1.00      1.00      1000

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000

[[1000    0]
 [   0 1000]]


In [ ]:
################################################################################
# Feature Importance - Random Forest
################################################################################

from sklearn.ensemble import RandomForestClassifier

n = 100

feature_importances = {}
for i in range(n):
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    sorted_features = sorted(zip(model.feature_importances_, model.feature_names_in_), reverse=True)
    for importance, name in sorted_features:
        if name in feature_importances:
            feature_importances[name].append(importance)
        else:
            feature_importances[name] = [importance]

average_feature_importances = {}
for name, importances in feature_importances.items():
    average_feature_importances[name] = sum(importances) / len(importances)

top_features = sorted(average_feature_importances.items(), key=lambda x: x[1], reverse=True)

with open(f"results/feature-importance-{sample_size}-rf.txt", "a") as f:
    f.write("\n".join([str(feature) for feature in top_features[:5]]))

print(top_features[:5])

In [6]:
################################################################################
# Predict from Logistic Regression model
################################################################################

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Create an instance of the DecisionTreeClassifier model
model = LogisticRegression()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict the labels for the test data
y_true = y_test
y_pred = model.predict(X_test)

# Evaluate the model
c_report = classification_report(y_true, y_pred)
c_matrix = confusion_matrix(y_true, y_pred)

with open(f"results/result-lr-{sample_size}-2.txt", "w") as f:
    f.write(f"Classication Report\n{c_report}\n\nConfusion Matrix\n{c_matrix}")

print(c_report)
print(c_matrix)

              precision    recall  f1-score   support

           0       0.67      0.80      0.73        10
           1       0.75      0.60      0.67        10

    accuracy                           0.70        20
   macro avg       0.71      0.70      0.70        20
weighted avg       0.71      0.70      0.70        20

[[8 2]
 [4 6]]


c:\Users\S4025371\OneDrive - RMIT University\Repositories\iot-llm\.conda\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
################################################################################
# Predict from SVM model
################################################################################

from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

# Create an instance of the DecisionTreeClassifier model
model = SVC()

# Fit the model to the training data
model.fit(X_train, y_train)

# Predict the labels for the test data
y_true = y_test
y_pred = model.predict(X_test)

# Evaluate the model
c_report = classification_report(y_true, y_pred)
c_matrix = confusion_matrix(y_true, y_pred)

with open(f"results/result-svm-{sample_size}-2.txt", "w") as f:
    f.write(f"Classication Report\n{c_report}\n\nConfusion Matrix\n{c_matrix}")

print(c_report)
print(c_matrix)

              precision    recall  f1-score   support

           0       0.62      0.50      0.56        10
           1       0.58      0.70      0.64        10

    accuracy                           0.60        20
   macro avg       0.60      0.60      0.60        20
weighted avg       0.60      0.60      0.60        20

[[5 5]
 [3 7]]
